# Libaries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from scipy import stats
import seaborn as sns

SEED = 187 # set seed for reproducibility


# Parameters

In [ ]:
# --------------------------------------------------
#  PARAMETERS 
# --------------------------------------------------
PRICE = 30.0            # revenue per unit sold
HOLDING_COST = 8.0      # cost per unit held one extra period
SCRAP_COST = 8.0        # cost per unit scrapped immediately
SHORTAGE_COST = 13.0     # penalty per unit unmet demand

# Number of batches (optional)
NUM_BATCHES = 1

INITIAL_INVENTORY = 100  # starting units per Batch
HORIZON = 50           # number of decision periods per episode
NUM_EPISODES = 50      # Monte Carlo repetitions
VECTOR_DIM = 2          # feature vector = [1, scaled_age]

# FRACTION (arms) f_t
SCRAP_FRACTIONS = np.linspace(0.0, 1.0, 99)  # can be adjusted 


# DGLM PARAMETERS (specified in emprircal setup, uncomment otherwise)
# --------------------------------------------------
# process_noises = [   
#     0.01,  # Batch 1
#     0.01,  # Batch 2
#     0.01,  # Batch 3
# ]

# For plotting
# --------------------------------------------------
t_min = 5.0  # minimum x axis
t_max = 15.0  # maximum x axis


# Demand Modeling

In [ ]:
# DEMAND
# Characteristics of the demand process
demand_specs = [
    {"batch": 1, "intercept": np.log(30),  "age_slope": -0.0},   # Seems to struggle with high demand numbers
    {"batch": 2, "intercept": np.log(30),  "age_slope": -0.0},  
    {"batch": 3, "intercept": np.log(30),  "age_slope": -0.0},
]
df_specs = pd.DataFrame(demand_specs)

# Store true betas for each batch
true_betas = df_specs[["intercept", "age_slope"]].values 

# Build true lambdas
true_lams_ex = np.zeros((NUM_BATCHES, HORIZON+1))
for b in range(NUM_BATCHES):
    for t in range(HORIZON+1):
        x = np.array([1.0, t / HORIZON])
        true_lams_ex[b, t] = np.exp(true_betas[b] @ x)
df_true_lams = pd.DataFrame(true_lams_ex)
df_true_lams.index.name = "Batch"


# Inspect
print("Demand Specs")
display(df_specs)
print("\nTrue Lambda Paths")
display(df_true_lams)

## Experiment Settings Grid

In [ ]:
# EPISODES
num_grid_episodes = 50


# Define the parameter grid
decay_rates = [-0.1, -2.5, -5.0]
process_noises = [0.01]
min_scrap_ages = [float(i) for i in range(1, 41)]  # minimum ages of scrap from 1 to 40

# For storing custom priors
custom_prior_means = {}
custom_prior_covs = {}

# Initialize with grid-specific priors
for decay_rate in decay_rates:
    for noise in process_noises:
        for scrap_age in min_scrap_ages:
            key = (decay_rate, noise, scrap_age)
            
            # Mean
            custom_prior_means[key] = [
                np.array([np.log(30), 0]),  # Batch 1 (intercept, decay rate)
                np.array([np.log(30), 0]),  # Batch 2 (intercept, decay rate)
                np.array([np.log(30), 0]),  # Batch 3 (intercept, decay rate)
            ]
            
            # Covariance
            base_var = 1  # Base variance
            cov_scale = max(1.0, 0)  # Scale factor based on process noise
            custom_prior_covs[key] = base_var * cov_scale * np.eye(2)

# Initialize dictionary to store results
grid_results = {}





# Economic Tradeoff

In [ ]:
# --------------------------------------------------
#  FOR EXPECTED SALES & PROFIT
# --------------------------------------------------
def expected_sales(k: int, lam: float) -> float:
    if lam <= 0:
        return 0.0
    lam_cap = min(lam, 1000.0) # caped to avoid overflow
    sales = sum(n * stats.poisson.pmf(n, lam_cap) for n in range(k + 1))
    tail = stats.poisson.sf(k, lam_cap)
    return sales + k * tail

def expected_future_profit(hold: int, lam: float) -> float:
    lam = float(np.clip(lam, 0.01, 1000.0))
    s = expected_sales(hold, lam)
    leftover = hold - s
    shortage = lam - s
    return s * PRICE - leftover * HOLDING_COST - shortage * SHORTAGE_COST

# Poisson-GLM

In [ ]:
# --------------------------------------------------
#  BAYESIAN POISSON DGLM (safeguarded against overflow)
# --------------------------------------------------
class BayesianPoissonDGLM:
    def __init__(self, dim: int, prior_var: float, process_noise: float, prior_mean=None):
        self.d = dim
        
        self.mu = prior_mean if prior_mean is not None else np.zeros(dim)
        
        if not isinstance(self.mu, np.ndarray):
            self.mu = np.array([self.mu] * dim if np.isscalar(self.mu) else self.mu)
            
        # Validate
        if len(self.mu) != dim:
            raise ValueError(f"Prior mean dim({len(self.mu)}) model dim ({dim})")
            
        if isinstance(prior_var, (int, float)):
            self.Sigma = prior_var * np.eye(dim)
        else:
            self.Sigma = prior_var
            
        self.W = process_noise * np.eye(dim)
        

    # Thompson sampling
    def sample_posterior(self, seed=None) -> np.ndarray: 
        # Local RNG for reproducibility
        if seed is not None:
            rng = np.random.default_rng(seed)
            return rng.multivariate_normal(self.mu, self.Sigma)
        else:
            return np.random.multivariate_normal(self.mu, self.Sigma)

    # Prediction of mean demand rate
    def predict_mean(self, x: np.ndarray, beta: np.ndarray = None) -> float:
        b = beta if beta is not None else self.mu
        eta = float(b @ x)
        eta = np.clip(eta, -1000.0, 1000.0)
        return float(np.exp(eta))

    # Update the model with new observation
    def update(self, x: np.ndarray, y: int) -> None:
        mu_p = self.mu.copy() 
        Sigma_p = self.Sigma + self.W # process noise !!

        eta_p = float(mu_p @ x)
        eta_p = np.clip(eta_p, -1000.0, 1000.0)
        lam_p = max(np.exp(eta_p), 1e-8)

        z = eta_p + (y - lam_p) / lam_p
        R = 1.0 / lam_p
        R = max(R, 0.001) # safeguard against division by zero

        H = x.reshape(1, -1)
        S = float((H @ Sigma_p @ H.T + R)[0][0])
        S = max(S, 1e-8)
        K = (Sigma_p @ H.T).flatten() / S

        self.mu = mu_p + K * (z - eta_p)
        self.mu = np.clip(self.mu, -1000.0, 1000.0)

        IKH = np.eye(self.d) - np.outer(K, H.flatten())
        self.Sigma = IKH @ Sigma_p @ IKH.T + 1e-6 * np.eye(self.d)
        eigs = np.linalg.eigvalsh(self.Sigma)
        if np.min(eigs) <= 0:
            self.Sigma += (abs(np.min(eigs)) + 1e-6) * np.eye(self.d)

# Simulation

In [ ]:
# --------------------------------------------------
#  SIMULATION OF ONE EPISODE
# --------------------------------------------------
def run_episode(seed=None, prior_means=None, prior_cov=None):
    if seed is not None:
        np.random.seed(seed)    # Sample each batch's initial demand rate path

    # Use default parameters if not provided
    if prior_means is None:
        # Create a default prior with zeros or appropriate values
        prior_means = [np.array([0.0, 0.0]) for _ in range(NUM_BATCHES)]

    if prior_cov is None:
        # Use a default covariance, perhaps 1.0 * identity matrix
        prior_cov = 1.0

    # --------------------------------------------------
    # DEMAND GENERATION

    true_betas = df_specs[["intercept", "age_slope"]].values
    true_lams  = np.zeros((NUM_BATCHES, HORIZON+1))
    for b in range(NUM_BATCHES):
        for t in range(HORIZON+1):
            x = np.array([1.0, t / HORIZON])
            true_lams[b, t] = np.exp(true_betas[b] @ x)


# --------------------------------------------------
    # Initialize inventory and models
    inventory = np.full(NUM_BATCHES, INITIAL_INVENTORY, dtype=int)
    models = []
    for b in range(NUM_BATCHES):
        if isinstance(prior_cov, np.ndarray) and prior_cov.ndim == 3:
            batch_cov = prior_cov[b]  # Extract just this batch's covariance
        else:
            batch_cov = prior_cov
            
        models.append(BayesianPoissonDGLM(
            VECTOR_DIM,
            batch_cov,
            process_noises[b],
            prior_mean=prior_means[b]
        ))
    
    # --------------------------------------------------
    #  ALLOCATE TRACKERS
    # --------------------------------------------------
    inv_pre    = np.zeros((NUM_BATCHES, HORIZON))
    dem_obs    = np.zeros((NUM_BATCHES, HORIZON))
    sales      = np.zeros((NUM_BATCHES, HORIZON))
    scr_frac   = np.zeros((NUM_BATCHES, HORIZON))
    oracle_f   = np.zeros((NUM_BATCHES, HORIZON))
    inv_post   = np.zeros((NUM_BATCHES, HORIZON))
    beta_draws = np.zeros((NUM_BATCHES, HORIZON, VECTOR_DIM))
    lam_hats   = np.zeros((NUM_BATCHES, HORIZON))
    mu_history = np.zeros((NUM_BATCHES, HORIZON+1, VECTOR_DIM))
    Sigma_history = np.zeros((NUM_BATCHES, HORIZON+1, VECTOR_DIM, VECTOR_DIM))
    
    
    # record initial prior
    for b in range(NUM_BATCHES):
        mu_history[b, 0, :] = models[b].mu
        Sigma_history[b, 0, :, :] = models[b].Sigma



    preds   = np.zeros((NUM_BATCHES, HORIZON))
    regrets = np.zeros((NUM_BATCHES, HORIZON))

    # to avoid indexing issues
    h = true_lams.shape[1] - 1  # number of decision periods

    # --------------------------------------------------
    #  MONTHLY LOOP
    # --------------------------------------------------
    for t in range(h):
        # 1) Record inventory at period start
        inv_pre[:, t] = inventory.copy()

        # Observe demands (still generate demands even during no-sales periods)
        demands = np.random.poisson(true_lams[:, t])

        # Demand learning phase
        if t < min_scrap_age:
            dem_obs[:, t] = demands  # Still observe demand for learning
            sales[:, t] = 0          # But record zero sales
            
            # Update model beliefs based on observed demand
            for b in range(NUM_BATCHES):
                x = np.array([1.0, t / HORIZON])
                models[b].update(x, demands[b])
                mu_history[b, t+1, :] = models[b].mu
                Sigma_history[b, t+1, :, :] = models[b].Sigma
                

                next_x = np.array([1.0, (t + 1) / HORIZON])
                preds[b, t] = models[b].predict_mean(next_x)
            
                regrets[b, t] = 0
                scr_frac[b, t] = 0
                oracle_f[b, t] = 0
                inv_post[b, t] = inventory[b]  # keep inventory to determine overstock directly

            # Skip the rest of the loop logic
            continue
        
        # Observe demands
        demands = np.random.poisson(true_lams[:, t])
        dem_obs[:, t] = demands

        for b in range(NUM_BATCHES):
            x = np.array([1.0, t / HORIZON])
            models[b].update(x, demands[b])
            mu_history[b, t+1, :] = models[b].mu
            Sigma_history[b, t+1, :, :] = models[b].Sigma

        # 3) Thompson sample one beta per batch
        beta_samps = [models[b].sample_posterior(seed=SEED + t * NUM_BATCHES + b) for b in range(NUM_BATCHES)]

        # 4) For each batch decide scrap fraction and record all metrics
        for b in range(NUM_BATCHES):
            # Actual sales
            sold = min(inventory[b], demands[b])
            sales[b, t] = sold

            # Revenue, shortage, leftover
            shortage = max(demands[b] - inventory[b], 0)
            revenue  = sold * PRICE
            leftover = inventory[b] - sold
            base     = revenue - shortage * SHORTAGE_COST

            # Next‐period context
            next_x = np.array([1.0, (t + 1) / HORIZON])

            # Track the sampled beta and the predicted lambda
            beta_draws[b, t, :] = beta_samps[b]
            lam_hats[b,    t]   = models[b].predict_mean(next_x, beta_samps[b])

            # Greedy evaluation over scrap fractions
            best_val = -math.inf
            best_f   = 0.0

            # Determine available scrap fractions
            available_fractions = [0.0] if t < min_scrap_age else SCRAP_FRACTIONS

            for f in available_fractions:
                hold = leftover - round(f * leftover)
                sc   = (leftover - hold) * SCRAP_COST
                hc   = hold * HOLDING_COST

                lam_hat = models[b].predict_mean(next_x, beta_samps[b])
                fut     = expected_future_profit(hold, lam_hat)
                val     = base - sc - hc + fut
                if val > best_val:
                    best_val = val
                    best_f   = f

            # Record chosen scrap fraction
            scr_frac[b, t] = best_f

            # Apply action
            hold = leftover - round(best_f * leftover)
            preds[b, t] = models[b].predict_mean(next_x)  # posterior‐mean forecast

            # 5) Compute regret
            lam_next = true_lams[b, t+1]
            agent_val = base \
                        - ((leftover - hold) * SCRAP_COST + hold * HOLDING_COST) \
                        + expected_future_profit(hold, lam_next)

            oracle_vals = []
            for f in available_fractions:
                h2   = leftover - round(f * leftover)
                sc2  = (leftover - h2) * SCRAP_COST
                hc2  = h2 * HOLDING_COST
                fut2 = expected_future_profit(h2, lam_next)
                oracle_vals.append(base - sc2 - hc2 + fut2)

            # Record oracle's scrap fraction and regret
            best_oracle_f    = SCRAP_FRACTIONS[np.argmax(oracle_vals)]
            oracle_f[b, t]   = best_oracle_f
            regrets[b, t]    = max(oracle_vals) - agent_val

            # 6) Update inventory and track post‐period level
            inventory[b]     = hold
            inv_post[b, t]   = hold

    return {
        "true_lams":  true_lams,
        "preds":      preds,
        "regrets":    regrets,
        "inv_pre":    inv_pre,
        "dem_obs":    dem_obs,
        "sales":      sales,
        "scr_frac":   scr_frac,
        "oracle_f":   oracle_f,
        "inv_post":   inv_post,
        "beta_draws": beta_draws,
        "lam_hats":   lam_hats,
        "mu_history": mu_history,
        "Sigma_history": Sigma_history
    }


# RUN EXPERIMENT


In [ ]:
min_scrap_age = 1.0 

# Grid search across all parameter combinations
for decay_rate in decay_rates:
    for noise in process_noises:
        for scrap_age in min_scrap_ages:
            key = (decay_rate, noise, scrap_age)
            print(f"\nRunning simulations for decay={decay_rate}, noise={noise}, min_scrap_age={scrap_age}")
            
            # Save original parameters
            original_df_specs = df_specs.copy()
            original_process_noises = process_noises.copy()
            original_min_scrap_age = min_scrap_age  # Store original value
            
            # Update parameters for this run
            updated_demand_specs = [
                {"batch": 1, "intercept": np.log(30), "age_slope": decay_rate},
                {"batch": 2, "intercept": np.log(30), "age_slope": decay_rate},
                {"batch": 3, "intercept": np.log(30), "age_slope": decay_rate}
            ]
            
            # Set parameters for this grid cell
            df_specs = pd.DataFrame(updated_demand_specs)
            true_betas = df_specs[["intercept", "age_slope"]].values
            process_noises = [noise] * NUM_BATCHES
            min_scrap_age = scrap_age 
            
            # Update true lambda values
            true_lams_ex = np.zeros((NUM_BATCHES, HORIZON+1))
            for b in range(NUM_BATCHES):
                for t in range(HORIZON+1):
                    x = np.array([1.0, t / HORIZON])
                    true_lams_ex[b, t] = np.exp(true_betas[b] @ x)
            
            # Run episodes for this parameter combination
            ep_results = []
            regrets_sum = np.zeros((NUM_BATCHES, HORIZON))
            rmse_sum = np.zeros(HORIZON)
            mae_sum = np.zeros(HORIZON)
            
            for ep in range(num_grid_episodes):
                result = run_episode(
                    seed=SEED + ep,  # Use different seed for each episode
                    prior_means=custom_prior_means[key],  # Prior mean
                    prior_cov=custom_prior_covs[key]      # Covariance
                )
                
                # Store result and calculate metrics
                ep_results.append(result)
                regrets_sum += result["regrets"]
                
                # Calculate RMSE and MAE
                true_lams = result["true_lams"][:, 1:]  # drop t=0
                preds = result["preds"]
                rmse = np.sqrt(((true_lams - preds) ** 2).mean(axis=0))
                mae = np.abs(true_lams - preds).mean(axis=0)
                rmse_sum += rmse
                mae_sum += mae
            
            # Calculate average metrics
            avg_regrets = regrets_sum / num_grid_episodes
            avg_rmse = rmse_sum / num_grid_episodes
            avg_mae = mae_sum / num_grid_episodes
            
            # Store results for THIS specific combination
            grid_results[key] = {
                "decay_rate": decay_rate,
                "process_noise": noise,
                "min_scrap_age": scrap_age,
                "avg_regret": avg_regrets.mean(),
                "cum_regret": avg_regrets.sum(),
                "avg_rmse": avg_rmse.mean(),
                "avg_mae": avg_mae.mean(),
                "regret_trajectory": avg_regrets.mean(axis=0),
                "rmse_trajectory": avg_rmse,
                "mae_trajectory": avg_mae,
                "episodes": ep_results
            }
            
            # Report key metrics
            print(f"  Average regret: {avg_regrets.mean():.4f}")
            print(f"  Cumulative regret: {avg_regrets.sum():.4f}")
            print(f"  Average RMSE: {avg_rmse.mean():.4f}")
            print(f"  Average MAE: {avg_mae.mean():.4f}")
            
            # Restore original parameters
            df_specs = original_df_specs
            process_noises = original_process_noises
            min_scrap_age = original_min_scrap_age  # Restore original value

## Metrics

In [ ]:
# Function to create scrapping analysis tables
def create_scrapping_tables(grid_results, decay_rates, min_scrap_ages, process_noise=0.01):
    tables = {}
    
    for decay_rate in decay_rates:
        for scrap_age in min_scrap_ages:
            key = (decay_rate, process_noise, scrap_age)
            if key not in grid_results:
                print(f"Missing data for {key}")
                continue
                
            episodes = grid_results[key]["episodes"]
            

            oracle_fractions = []
            agent_fractions = []
            cumulative_regrets = []
            units_scrapped = []
            inv_pres = []
            sales_list = []
            calculated_scrap = []
            

            for ep in episodes:

                oracle_f = np.mean(ep["oracle_f"], axis=0)
                agent_f = np.mean(ep["scr_frac"], axis=0)
                regrets = np.mean(ep["regrets"], axis=0)
                cum_regret = np.cumsum(regrets)
                
                inv_pre = np.mean(ep["inv_pre"], axis=0)
                inv_post = np.mean(ep["inv_post"], axis=0)
                ep_sales = np.mean(ep["sales"], axis=0)
                
                scrapped = inv_pre - inv_post
                calc_scrap = (inv_pre - ep_sales) * agent_f
                
                oracle_fractions.append(oracle_f)
                agent_fractions.append(agent_f)
                cumulative_regrets.append(cum_regret)
                units_scrapped.append(scrapped)
                inv_pres.append(inv_pre)
                sales_list.append(ep_sales)
                calculated_scrap.append(calc_scrap)
            
            # Average across episodes
            avg_oracle = np.mean(oracle_fractions, axis=0)
            avg_agent = np.mean(agent_fractions, axis=0)
            avg_cum_regret = np.mean(cumulative_regrets, axis=0)
            avg_units_scrapped = np.mean(units_scrapped, axis=0)
            avg_inv_pre = np.mean(inv_pres, axis=0)
            avg_sales = np.mean(sales_list, axis=0)
            avg_calc_scrap = np.mean(calculated_scrap, axis=0)
            
            time_steps = np.arange(len(avg_oracle))
            
            start_idx = int(scrap_age)
            relevant_steps = range(start_idx, min(start_idx + 10, len(time_steps)))
            
            data = {
                'Time Step': time_steps[relevant_steps],
                'Oracle Scrap Fraction': np.round(avg_oracle[relevant_steps], 2),
                'Agent Scrap Fraction': np.round(avg_agent[relevant_steps], 2),
                'Cumulative Regret': np.round(avg_cum_regret[relevant_steps], 2),
                'Inventory Before': np.round(avg_inv_pre[relevant_steps], 2),
                'Sales': np.round(avg_sales[relevant_steps], 2),
                'Calculated Units Scrapped': np.round(avg_calc_scrap[relevant_steps], 2)
            }
            
            tables[(decay_rate, scrap_age)] = pd.DataFrame(data)
            
    return tables


scrapping_tables = create_scrapping_tables(
    grid_results, 
    decay_rates=[-0.1, -2.5, -5.0], 
    min_scrap_ages=[10.0, 20.0, 40.0]
)

for (decay, scrap_age), table in scrapping_tables.items():
    print(f"\n=== Decay Rate: {decay}, Min Scrap Age: {scrap_age} ===")
    display(table)

    with open("scrapping_tables_100_1-40.txt", "w") as f:
        for (decay, scrap_age), table in scrapping_tables.items():
            f.write(f"=== Decay Rate: {decay}, Min Scrap Age: {scrap_age} ===\n")
            f.write(table.to_string())
            f.write("\n\n")

In [ ]:
# --------------------------------------------------
#  PLOTTING FIRST SCRAP FRACTIONS
# --------------------------------------------------
first_scrap_data = []
for decay_rate in decay_rates:
    for scrap_age in min_scrap_ages:
        key = (decay_rate, process_noises[0], scrap_age)
        episodes = grid_results[key]["episodes"]
        oracle_first_scraps = []
        agent_first_scraps = []
        for ep in episodes:
            scrap_start_idx = int(scrap_age)
            oracle_first_scrap = np.mean(ep["oracle_f"][:, scrap_start_idx])
            agent_first_scrap = np.mean(ep["scr_frac"][:, scrap_start_idx])
            oracle_first_scraps.append(oracle_first_scrap)
            agent_first_scraps.append(agent_first_scrap)
        avg_oracle_first = np.mean(oracle_first_scraps)
        avg_agent_first = np.mean(agent_first_scraps)
        first_scrap_data.append({
            "Decay Rate": decay_rate,
            "Min Scrap Age": scrap_age,
            "Decision Maker": "Oracle",
            "First Scrap Fraction": avg_oracle_first
        })
        first_scrap_data.append({
            "Decay Rate": decay_rate,
            "Min Scrap Age": scrap_age,
            "Decision Maker": "Agent",
            "First Scrap Fraction": avg_agent_first
        })

first_scrap_df = pd.DataFrame(first_scrap_data)
positions = np.arange(len(min_scrap_ages))
fig = plt.figure(figsize=(14, 8))
width = 0.15

for i, decay_rate in enumerate(decay_rates):
    subset = first_scrap_df[first_scrap_df["Decay Rate"] == decay_rate]
    oracle_vals = []
    agent_vals = []
    for age in min_scrap_ages:
        age_subset = subset[subset["Min Scrap Age"] == age]
        oracle_vals.append(age_subset[age_subset["Decision Maker"] == "Oracle"]["First Scrap Fraction"].values[0])
        agent_vals.append(age_subset[age_subset["Decision Maker"] == "Agent"]["First Scrap Fraction"].values[0])
    plt.bar(positions + (i*2-1)*width, oracle_vals, width=width, 
            label=f"Oracle (Decay={decay_rate})", 
            color=plt.cm.viridis(i/len(decay_rates)), alpha=0.8)
    plt.bar(positions + (i*2)*width, agent_vals, width=width, 
            label=f"Bandit (Decay={decay_rate})", 
            color=plt.cm.viridis(i/len(decay_rates)), alpha=0.5, hatch="//")

plt.xticks(positions, [f"{int(a)}" for a in min_scrap_ages], fontsize=12)
plt.xlabel('Age at First Scrap', fontsize=14)
plt.ylabel('First Scrap Fraction', fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.legend(fontsize=10, loc='best')
plt.tight_layout()
plt.savefig("first_scrap_fractions_100_1-40_1.png", dpi=300, bbox_inches='tight')
plt.show()


# Generating new stats

- "run_episode" function gnereates experiment data
- grid_results stores the results of the different parameters tested
- "return" command shows, where to allocate metrics